# 간단한 RAG에서의 컨텍스트 청크 헤더(CCH)

검색 증강 생성(RAG)은 응답을 생성하기 전에 관련 외부 지식을 검색하여 언어 모델의 사실적 정확성을 향상시킵니다. 그러나 표준적인 청킹 방식은 종종 중요한 컨텍스트를 잃어버려 검색 효율성을 떨어뜨립니다.

컨텍스트 청크 헤더(CCH)는 각 청크를 임베딩하기 전에 문서 제목이나 섹션 헤더와 같은 상위 수준의 컨텍스트를 앞에 추가하여 RAG를 향상시킵니다. 이는 검색 품질을 향상시키고 문맥에서 벗어난 응답을 방지합니다.

> 💡 **초보자를 위한 설명:** 일반적인 RAG는 책의 내용을 페이지 단위로 잘라서 필요할 때 찾아보는 것과 같습니다. 하지만 이렇게 하면 각 페이지가 어떤 장에 속해 있었는지 잊어버리기 쉽습니다. '컨텍스트 청크 헤더'는 각 페이지(청크) 앞에 '1장: 인공지능의 역사'와 같은 장 제목(헤더)을 붙여주는 것과 같습니다. 이렇게 하면 AI가 검색할 때 각 내용 조각이 어떤 맥락에 속하는지 더 잘 이해하게 되어, 훨씬 더 정확하고 관련성 높은 정보를 찾을 수 있습니다.

## 이 노트북의 단계:

1. **데이터 수집**: 텍스트 데이터를 로드하고 전처리합니다.
2. **컨텍스트 헤더를 사용한 청킹**: 섹션 제목을 추출하여 청크 앞에 추가합니다.
3. **임베딩 생성**: 컨텍스트가 강화된 청크를 숫자 표현으로 변환합니다.
4. **시맨틱 검색**: 사용자 쿼리를 기반으로 관련 청크를 검색합니다.
5. **응답 생성**: 검색된 텍스트를 사용하여 언어 모델로 응답을 생성합니다.
6. **평가**: 채점 시스템을 사용하여 응답 정확도를 평가합니다.

## 환경 설정
필요한 라이브러리를 가져오는 것으로 시작하겠습니다.

In [1]:
import os
import numpy as np
import json
from openai import OpenAI
import fitz
from tqdm import tqdm

## 텍스트 추출 및 섹션 헤더 식별
PDF에서 텍스트를 추출하면서 섹션 제목(청크의 잠재적 헤더)도 식별합니다.

In [2]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
    pdf_path (str): PDF 파일 경로.

    Returns:
    str: PDF에서 추출된 텍스트.
    """
    # PDF 파일을 엽니다
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 빈 문자열을 초기화합니다

    # PDF의 각 페이지를 반복합니다
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 페이지를 가져옵니다
        text = page.get_text("text")  # 페이지에서 텍스트를 추출합니다
        all_text += text  # 추출된 텍스트를 all_text 문자열에 추가합니다

    return all_text  # 추출된 텍스트를 반환합니다

## OpenAI API 클라이언트 설정
임베딩과 응답을 생성하기 위해 OpenAI 클라이언트를 초기화합니다.

In [ ]:
# 기본 URL과 API 키로 OpenAI 클라이언트를 초기화합니다
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키를 가져옵니다
)

## 컨텍스트 헤더를 사용한 텍스트 청킹
검색 성능을 향상시키기 위해 LLM 모델을 사용하여 각 청크에 대한 설명적인 헤더를 생성합니다.

> 💡 **초보자를 위한 설명:** 이 단계에서는 단순히 기존 문서의 제목을 가져오는 것을 넘어, LLM을 시켜 각 텍스트 조각의 내용을 요약하는 새로운 '미니 제목(헤더)'을 만들어냅니다. 예를 들어, 인공지능의 윤리에 대한 긴 단락이 있다면, LLM이 "AI 윤리의 중요성과 편향 문제"와 같은 간결한 헤더를 생성해줍니다. 이 헤더는 원본 텍스트와 함께 임베딩되어, 검색 시 더 정확한 문맥 정보를 제공하는 역할을 합니다.

In [4]:
def generate_chunk_header(chunk, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    LLM을 사용하여 주어진 텍스트 청크에 대한 제목/헤더를 생성합니다.

    Args:
    chunk (str): 헤더로 요약할 텍스트 청크.
    model (str): 헤더 생성에 사용할 모델. 기본값은 "meta-llama/Llama-3.2-3B-Instruct"입니다.

    Returns:
    str: 생성된 헤더/제목.
    """
    # AI의 행동을 안내하는 시스템 프롬프트를 정의합니다
    system_prompt = "주어진 텍스트에 대해 간결하고 정보성 있는 제목을 생성하세요."
    
    # 시스템 프롬프트와 텍스트 청크를 기반으로 AI 모델로부터 응답을 생성합니다
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": chunk}
        ]
    )

    # 생성된 헤더/제목을 반환하되, 앞뒤 공백을 제거합니다
    return response.choices[0].message.content.strip()

In [5]:
def chunk_text_with_headers(text, n, overlap):
    """
    텍스트를 더 작은 세그먼트로 나누고 헤더를 생성합니다.

    Args:
    text (str): 청크로 나눌 전체 텍스트.
    n (int): 청크 크기(문자 수).
    overlap (int): 청크 간의 중첩 문자 수.

    Returns:
    List[dict]: 'header'와 'text' 키를 가진 딕셔너리 리스트.
    """
    chunks = []  # 청크를 저장할 빈 리스트를 초기화합니다

    # 지정된 청크 크기와 중첩으로 텍스트를 반복합니다
    for i in range(0, len(text), n - overlap):
        chunk = text[i:i + n]  # 텍스트 청크를 추출합니다
        header = generate_chunk_header(chunk)  # LLM을 사용하여 청크에 대한 헤더를 생성합니다
        chunks.append({"header": header, "text": chunk})  # 헤더와 청크를 리스트에 추가합니다

    return chunks  # 헤더가 있는 청크 리스트를 반환합니다

## PDF 파일에서 텍스트 추출 및 청킹
이제 PDF를 로드하고, 텍스트를 추출한 다음, 청크로 분할합니다.

In [6]:
# PDF 파일 경로를 정의합니다
pdf_path = "data/AI_Information.pdf"

# PDF 파일에서 텍스트를 추출합니다
extracted_text = extract_text_from_pdf(pdf_path)

# 헤더와 함께 추출된 텍스트를 청킹합니다
# 청크 크기는 1000자, 중첩은 200자로 사용합니다
text_chunks = chunk_text_with_headers(extracted_text, 1000, 200)

# 생성된 헤더와 함께 샘플 청크를 출력합니다
print("Sample Chunk:")
print("Header:", text_chunks[0]['header']) 
print("Content:", text_chunks[0]['text'])

Sample Chunk:
Header: "Introduction to Artificial Intelligence: Understanding the Foundations and Evolution"
Content: Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the development and deployment of AI. 
Historical Context 
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop 
in 1956 is widely considered

## 헤더와 텍스트에 대한 임베딩 생성
검색 정확도를 높이기 위해 헤더와 텍스트 모두에 대한 임베딩을 생성합니다.

In [7]:
def create_embeddings(text, model="BAAI/bge-en-icl"):
    """
    주어진 텍스트에 대한 임베딩을 생성합니다.

    Args:
    text (str): 임베딩할 입력 텍스트.
    model (str): 사용할 임베딩 모델. 기본값은 "BAAI/bge-en-icl"입니다.

    Returns:
    dict: 입력 텍스트에 대한 임베딩을 포함하는 응답.
    """
    # 지정된 모델과 입력 텍스트를 사용하여 임베딩을 생성합니다
    response = client.embeddings.create(
        model=model,
        input=text
    )
    # 응답에서 임베딩을 반환합니다
    return response.data[0].embedding

In [8]:
# 각 청크에 대한 임베딩을 생성합니다
embeddings = []  # 임베딩을 저장할 빈 리스트를 초기화합니다

# 진행률 표시줄과 함께 각 텍스트 청크를 반복합니다
for chunk in tqdm(text_chunks, desc="Generating embeddings"):
    # 청크의 텍스트에 대한 임베딩을 생성합니다
    text_embedding = create_embeddings(chunk["text"])
    # 청크의 헤더에 대한 임베딩을 생성합니다
    header_embedding = create_embeddings(chunk["header"])
    # 청크의 헤더, 텍스트 및 해당 임베딩을 리스트에 추가합니다
    embeddings.append({"header": chunk["header"], "text": chunk["text"], "embedding": text_embedding, "header_embedding": header_embedding})

Generating embeddings: 100%|██████████| 42/42 [02:56<00:00,  4.21s/it]


## 시맨틱 검색 수행
사용자 쿼리에 가장 관련성 높은 텍스트 청크를 찾기 위해 코사인 유사도를 구현합니다.

> 💡 **초보자를 위한 설명:** 검색 시, 우리는 질문(쿼리)과 (1)청크의 원본 내용, (2)청크의 요약 헤더 각각의 유사도를 계산합니다. 그런 다음 이 두 유사도 점수의 평균을 내어 최종 관련성을 판단합니다. 이렇게 하면 원본 내용의 세부 정보와 헤더의 핵심 요약 정보를 모두 활용할 수 있어, 검색 정확도가 크게 향상됩니다.

In [9]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터.
    vec2 (np.ndarray): 두 번째 벡터.

    Returns:
    float: 코사인 유사도 점수.
    """
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [10]:
def semantic_search(query, chunks, k=5):
    """
    쿼리를 기반으로 가장 관련성 높은 청크를 검색합니다.

    Args:
    query (str): 사용자 쿼리.
    chunks (List[dict]): 임베딩이 포함된 텍스트 청크 리스트.
    k (int): 상위 결과 수.

    Returns:
    List[dict]: 상위 k개의 가장 관련성 높은 청크.
    """
    # 쿼리에 대한 임베딩을 생성합니다
    query_embedding = create_embeddings(query)

    similarities = []  # 유사도 점수를 저장할 리스트를 초기화합니다
    
    # 각 청크를 반복하여 유사도 점수를 계산합니다
    for chunk in chunks:
        # 쿼리 임베딩과 청크 텍스트 임베딩 간의 코사인 유사도를 계산합니다
        sim_text = cosine_similarity(np.array(query_embedding), np.array(chunk["embedding"]))
        # 쿼리 임베딩과 청크 헤더 임베딩 간의 코사인 유사도를 계산합니다
        sim_header = cosine_similarity(np.array(query_embedding), np.array(chunk["header_embedding"]))
        # 평균 유사도 점수를 계산합니다
        avg_similarity = (sim_text + sim_header) / 2
        # 청크와 평균 유사도 점수를 리스트에 추가합니다
        similarities.append((chunk, avg_similarity))

    # 유사도 점수를 기준으로 청크를 내림차순으로 정렬합니다
    similarities.sort(key=lambda x: x[1], reverse=True)
    # 상위 k개의 가장 관련성 높은 청크를 반환합니다
    return [x[0] for x in similarities[:k]]

## 추출된 청크에 대해 쿼리 실행하기

In [11]:
# 검증 데이터를 로드합니다
with open('data/val.json') as f:
    data = json.load(f)

query = data[0]['question']

# 가장 관련성 높은 상위 2개 텍스트 청크를 검색합니다
top_chunks = semantic_search(query, embeddings, k=2)

# 결과를 출력합니다
print("Query:", query)
for i, chunk in enumerate(top_chunks):
    print(f"Header {i+1}: {chunk['header']}")
    print(f"Content:
{chunk['text']}
")

Query: What is 'Explainable AI' and why is it considered important?
Header 1: "Building Trust in AI: Addressing Transparency, Explainability, and Accountability"
Content:
systems. Explainable AI (XAI) 
techniques aim to make AI decisions more understandable, enabling users to assess their 
fairness and accuracy. 
Privacy and Data Protection 
AI systems often rely on large amounts of data, raising concerns about privacy and data 
protection. Ensuring responsible data handling, implementing privacy-preserving techniques, 
and complying with data protection regulations are crucial. 
Accountability and Responsibility 
Establishing accountability and responsibility for AI systems is essential for addressing potential 
harms and ensuring ethical behavior. This includes defining roles and responsibilities for 
developers, deployers, and users of AI systems. 
Chapter 20: Building Trust in AI 
Transparency and Explainability 
Transparency and explainability are key to building trust in AI. Maki

## 검색된 청크를 기반으로 응답 생성하기

In [12]:
# AI 어시스턴트에 대한 시스템 프롬프트를 정의합니다
system_prompt = "당신은 주어진 컨텍스트를 기반으로만 엄격하게 답변하는 AI 어시스턴트입니다. 제공된 컨텍스트에서 직접 답변을 도출할 수 없는 경우, '답변하기에 충분한 정보가 없습니다.'라고 응답하세요."

def generate_response(system_prompt, user_message, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    시스템 프롬프트와 사용자 메시지를 기반으로 AI 모델로부터 응답을 생성합니다.

    Args:
    system_prompt (str): AI의 행동을 안내하는 시스템 프롬프트.
    user_message (str): 사용자의 메시지 또는 쿼리.
    model (str): 응답 생성에 사용할 모델. 기본값은 "meta-llama/Llama-2-7B-chat-hf"입니다.

    Returns:
    dict: AI 모델의 응답.
    """
    response = client.chat.com.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# 상위 청크를 기반으로 사용자 프롬프트를 생성합니다
user_prompt = "\n".join([f"Header: {chunk['header']}\nContent:\n{chunk['text']}" for chunk in top_chunks])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# AI 응답을 생성합니다
ai_response = generate_response(system_prompt, user_prompt)

## AI 응답 평가하기
AI의 응답을 기대 답변과 비교하여 점수를 매깁니다.

In [13]:
# 평가 시스템 프롬프트를 정의합니다
evaluate_system_prompt = """당신은 지능형 평가 시스템입니다. 
Assess the AI assistant's response based on the provided context. 
- Assign a score of 1 if the response is very close to the true answer. 
- Assign a score of 0.5 if the response is partially correct. 
- Assign a score of 0 if the response is incorrect.
Return only the score (0, 0.5, or 1)."""

# 검증 데이터에서 정답을 추출합니다
true_answer = data[0]['ideal_answer']

# 평가 프롬프트를 구성합니다
evaluation_prompt = f"""
User Query: {query}
AI Response: {ai_response}
True Answer: {true_answer}
{evaluate_system_prompt}
"""

# 평가 점수를 생성합니다
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 평가 점수를 출력합니다
print("Evaluation Score:", evaluation_response.choices[0].message.content)

Evaluation Score: 0.5
